In [1]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import lightning as L
from trainer import LitModule, wandb_logger, profiler

/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/accelerator_connector.py:552: You passed `Trainer(accelerator='cpu', precision='16-mixed')` but AMP with fp16 is not supported on CPU. Using `precision='bf16-mixed'` instead.
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [3]:
os.getenv("WANDB_API_KEY")

In [4]:
def check_model_w(model):
    # Access the model's state dictionary
    state_dict = model.state_dict()

    # Print the keys and shapes of the model's parameters
    for key, value in state_dict.items():
        print(f"Parameter: {key}, Size: {value.size()}")

    # Alternatively, you can directly print the state_dict
    print(state_dict)

In [5]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))

    def forward(self, x):
        return self.l1(x)


class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))

    def forward(self, x):
        return self.l1(x)


class TESTAUTOENCODER(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x, y):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn as nn

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        return sample.view(-1), sample.view(-1)  # Reshape to match the input and output size

# Generate sample data
num_samples = 1000
input_dim = 28 * 28  # Adjusted to match the input size of the Encoder

# Create synthetic data (random numbers)
data = torch.randn(num_samples, input_dim)

# Create an instance of the CustomDataset
dataset = CustomDataset(data)

# Define batch size for DataLoader
batch_size = 64

# Create a DataLoader
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Test the DataLoader by iterating through a few batches
for batch_idx, (inputs, targets) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}:")
    print("Input shape:", inputs.shape)
    print("Target shape:", targets.shape)
    break  # Stop after the first batch for demonstration


Batch 1:
Input shape: torch.Size([64, 784])
Target shape: torch.Size([64, 784])


In [7]:
test_model = TESTAUTOENCODER()
check_model_w(test_model)

Parameter: encoder.l1.0.weight, Size: torch.Size([64, 784])
Parameter: encoder.l1.0.bias, Size: torch.Size([64])
Parameter: encoder.l1.2.weight, Size: torch.Size([3, 64])
Parameter: encoder.l1.2.bias, Size: torch.Size([3])
Parameter: decoder.l1.0.weight, Size: torch.Size([64, 3])
Parameter: decoder.l1.0.bias, Size: torch.Size([64])
Parameter: decoder.l1.2.weight, Size: torch.Size([784, 64])
Parameter: decoder.l1.2.bias, Size: torch.Size([784])
OrderedDict([('encoder.l1.0.weight', tensor([[-0.0161,  0.0290, -0.0210,  ...,  0.0026,  0.0268,  0.0169],
        [-0.0016, -0.0153, -0.0268,  ...,  0.0312,  0.0130, -0.0154],
        [ 0.0150, -0.0332,  0.0200,  ...,  0.0074,  0.0344, -0.0218],
        ...,
        [-0.0328,  0.0232,  0.0154,  ..., -0.0357,  0.0335, -0.0088],
        [-0.0296, -0.0015, -0.0194,  ...,  0.0030,  0.0158,  0.0228],
        [ 0.0085,  0.0144, -0.0004,  ..., -0.0089,  0.0054,  0.0033]])), ('encoder.l1.0.bias', tensor([ 0.0029, -0.0243,  0.0221, -0.0050, -0.0082, -0.0

In [8]:
# model
MODEL_ = LitModule(model=test_model, loss_criterion=F.mse_loss, metric=None)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: rileydrizzy. Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [9]:
check_model_w(MODEL_)

Parameter: model.encoder.l1.0.weight, Size: torch.Size([64, 784])
Parameter: model.encoder.l1.0.bias, Size: torch.Size([64])
Parameter: model.encoder.l1.2.weight, Size: torch.Size([3, 64])
Parameter: model.encoder.l1.2.bias, Size: torch.Size([3])
Parameter: model.decoder.l1.0.weight, Size: torch.Size([64, 3])
Parameter: model.decoder.l1.0.bias, Size: torch.Size([64])
Parameter: model.decoder.l1.2.weight, Size: torch.Size([784, 64])
Parameter: model.decoder.l1.2.bias, Size: torch.Size([784])
OrderedDict([('model.encoder.l1.0.weight', tensor([[-0.0161,  0.0290, -0.0210,  ...,  0.0026,  0.0268,  0.0169],
        [-0.0016, -0.0153, -0.0268,  ...,  0.0312,  0.0130, -0.0154],
        [ 0.0150, -0.0332,  0.0200,  ...,  0.0074,  0.0344, -0.0218],
        ...,
        [-0.0328,  0.0232,  0.0154,  ..., -0.0357,  0.0335, -0.0088],
        [-0.0296, -0.0015, -0.0194,  ...,  0.0030,  0.0158,  0.0228],
        [ 0.0085,  0.0144, -0.0004,  ..., -0.0089,  0.0054,  0.0033]])), ('model.encoder.l1.0.bias

In [10]:
# train model
trainer = L.Trainer(
    accelerator="auto",
    precision="16-mixed",
    logger=wandb_logger,
    profiler=profiler,
    strategy="auto",
    check_val_every_n_epoch=5,
    max_epochs=15
)

trainer.fit(
    model=MODEL_,
    train_dataloaders=train_loader,
    val_dataloaders=train_loader, ckpt_path = None)

/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/accelerator_connector.py:552: You passed `Trainer(accelerator='cpu', precision='16-mixed')` but AMP with fp16 is not supported on CPU. Using `precision='bf16-mixed'` instead.
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:44: attribute 'model' removed from hparams because it cannot be pickled
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory /workspace/NSL_2_AUDIO/signa2text/src/artifacts/test exists and is

/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (16) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 14: 100%|██████████| 16/16 [00:01<00:00, 10.84it/s, v_num=kxtp]

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|██████████| 16/16 [00:01<00:00, 10.82it/s, v_num=kxtp]


# BREAK

In [11]:
check_model_w(MODEL_)

Parameter: model.encoder.l1.0.weight, Size: torch.Size([64, 784])
Parameter: model.encoder.l1.0.bias, Size: torch.Size([64])
Parameter: model.encoder.l1.2.weight, Size: torch.Size([3, 64])
Parameter: model.encoder.l1.2.bias, Size: torch.Size([3])
Parameter: model.decoder.l1.0.weight, Size: torch.Size([64, 3])
Parameter: model.decoder.l1.0.bias, Size: torch.Size([64])
Parameter: model.decoder.l1.2.weight, Size: torch.Size([784, 64])
Parameter: model.decoder.l1.2.bias, Size: torch.Size([784])
OrderedDict([('model.encoder.l1.0.weight', tensor([[-0.0210,  0.0298, -0.0125,  ...,  0.0038,  0.0356,  0.0230],
        [-0.0032, -0.0128, -0.0298,  ...,  0.0432,  0.0109, -0.0089],
        [ 0.0122, -0.0500,  0.0212,  ...,  0.0224,  0.0129, -0.0353],
        ...,
        [-0.0350,  0.0299,  0.0092,  ..., -0.0325,  0.0414,  0.0038],
        [-0.0198,  0.0006, -0.0258,  ..., -0.0216,  0.0161,  0.0072],
        [ 0.0133,  0.0207, -0.0245,  ..., -0.0178,  0.0005,  0.0099]])), ('model.encoder.l1.0.bias